In [12]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from pathlib import Path
from pprint import pprint

In [1]:
from dotenv import load_dotenv

load_dotenv('./.env')


True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
os.environ['LANGCHAIN_PROJECT'] = 'CPCP_test1'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[API KEY]
sk-l0rAOwEsqXqeXoZsj4jppfpH52TOFQezaNohE3ztURT3BlbkFJmqnZl5n_izzgDQoGAxMjXYm0nj4BrEzXWrArHJABUA
[LANGCHAIN_PROJECT]
CPCP_test1


In [5]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CPCP_test1")

LangSmith 추적을 시작합니다.
[프로젝트명]
CPCP_test1


In [69]:
from langchain_community.document_loaders import JSONLoader



# JSONLoader 생성
loader = JSONLoader(
    file_path="LAB_21대_제382회_제5차_본회의_국회본회의_0003(050187).json",
    jq_schema=".date",
    text_content=False,
)

# 문서 로드
docs = loader.load()

# 결과 출력
print(docs[0])


page_content='2020년9월15일(화)' metadata={'source': '/home/elicer/ChatBot_Project/practice_sm/reference/LAB_21대_제382회_제5차_본회의_국회본회의_0003(050187).json', 'seq_num': 1}


In [63]:
import json

# JSON 파일 읽기
with open('LAB_21대_제382회_제5차_본회의_국회본회의_0003(050187).json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# key 값 추출
keys_list = list(data.keys())

# 리스트 출력
print(keys_list)


['filename', 'original', 'id', 'date', 'conference_number', 'question_number', 'meeting_name', 'generation_number', 'committee_name', 'meeting_number', 'session_number', 'agenda', 'law', 'qna_type', 'context', 'context_learn', 'context_summary', 'questioner_name', 'questioner_ID', 'questioner_ISNI', 'questioner_affiliation', 'questioner_position', 'question', 'answerer_name', 'answerer_ID', 'answerer_ISNI', 'answerer_affiliation', 'answerer_position', 'answer']


In [79]:
from langchain.schema import Document

# 문서 리스트 생성
documents = []
for key, value in data.items():
    if isinstance(value, str):  # 값이 문자열인지 확인
        documents.append(Document(page_content=value, metadata={"key": key}))

# 생성된 문서 출력
for doc in documents:
    print(doc)


page_content='SRC_21대_제382회_제5차_본회의_국회본회의_0003(050187).xlsx' metadata={'key': 'filename'}
page_content='http://likms.assembly.go.kr/record/mhs-10-040-0040.do?conferNum=050187&fileType=PDF' metadata={'key': 'original'}
page_content='050187' metadata={'key': 'id'}
page_content='2020년9월15일(화)' metadata={'key': 'date'}
page_content='050187' metadata={'key': 'conference_number'}
page_content='0003' metadata={'key': 'question_number'}
page_content='본회의' metadata={'key': 'meeting_name'}
page_content='21' metadata={'key': 'generation_number'}
page_content='국회본회의' metadata={'key': 'committee_name'}
page_content='제382회' metadata={'key': 'meeting_number'}
page_content='제5차' metadata={'key': 'session_number'}
page_content='1. 외교·통일·안보에 관한 질문' metadata={'key': 'agenda'}
page_content='외교·통일·안보에 관한 질문' metadata={'key': 'law'}
page_content='추출형' metadata={'key': 'qna_type'}
page_content='한반도와 국제정세에 영향을 미칠 미국 대선이 두 달도 안 남았습니다. 외교부는 트럼프와 바이든 후보의 대선 공약과 정책을 구체적으로 파악하고 계시지요? 예, 외교부에 1차관 주도로 TF를 만들어서 동향을 면

In [85]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [86]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [87]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [88]:
from langchain_teddynote.messages import stream_response

In [94]:
answer = rag_chain.stream("21대 제5차 국회본회의의 주제는?")
# # 제너레이터에서 데이터를 하나씩 모아서 문자열로 합치기
# answer_content = ''.join([chunk for chunk in answer])

# # 결과 출력
# print(answer_content)
stream_response(answer)


21대 제5차 국회본회의의 주제는 '외교·통일·안보에 관한 질문'입니다.